In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
import os
import sys
import re

sys.path.append('../')

In [63]:
#variation_template = pd.read_csv('../data/variation_template_v2.csv')

variation_template = pd.read_csv('../data/variation_template_v3.csv')
data_df = pd.read_csv('../data/courses-data-btech-v4.csv')
output_dir = '../data/product_output/'

VARIATION_NAME = "Name"
VARIATION_SKU = "SKU"
VARIATION_IMAGE = "Images"
VARIATION_PARENT = "Parent"
VARIATION_CATEGORY = "Categories"
VARIATION_DESCRIPTION = "Description"

FLOAT_TO_INT = ['Meta: _et_pb_gutter_width']
PRODUCT_CODE_COL = 'aff_course_code'
AFFILIATION_BRANCH_NAME = 'aff_branch_names'
SKU_COLS = ['aff_name', 'aff_course_code']
NAME_COLS = [PRODUCT_CODE_COL, '2l_course_name']
PRODUCT_VIDEO_COLS = ['almvid', 'psmvid', 'gpsmvid']
PRODUCT_IMAGE_COL = 'course_image'
SEARCH_FORM = "{{%term}}"
FACULTY_COLS = ['Name_Fac{0}', 'Pic_Fac{0}', 'Desc_Fac{0}']
FAC_REPLACE = ["professor", "faculty_image", "prof description"]

IMAGE_REPLACEMENTS = [("http://test.2learn.in/wp-content/uploads/2023/01/Abhishek-sir.png", "{{faculty_image}}"),
                      ("http://test.2learn.in/wp-content/uploads/2023/05/image4.png", "{{Syl_Img}}")]
                      

# Key div changes. This will need to be replaced every time a change is made to divi product-template
VIDEO_SLIDER_DIV = '[et_pb_video_slider_item src="https://www.youtube.com/watch?v=FkQuawiGWUw" _builder_version="4.19.0" _module_preset="default" global_colors_info="{}" show_image_overlay="off"][/et_pb_video_slider_item]'
VIDEO_SLIDER_REPLACE = '[et_pb_video_slider_item src="{{video}}" _builder_version="4.19.0" _module_preset="default" global_colors_info="{}" show_image_overlay="off"][/et_pb_video_slider_item]'
FACULTY_DIV =  '[et_pb_column type="1_3" _builder_version="4.16" custom_padding="|||" global_colors_info="{}" custom_padding__hover="|||"][et_pb_team_member name="{{professor}}" image_url="{{faculty_image}}" facebook_url="#" twitter_url="#" google_url="#" linkedin_url="#" icon_color="RGBA(255,255,255,0)" use_icon_font_size="on" icon_font_size="1px" _builder_version="4.19.0" header_font="Archivo|700|||||||" header_text_color="#544FFF" header_font_size="18px" header_line_height="1.7em" body_font="||||||||" body_line_height="1.8em" background_color="#ffffff" text_orientation="center" custom_margin="||0px|" custom_padding="30px|3.7vw|0px|3.7vw|false|true" custom_padding_tablet="|17vw||17vw|false|true" custom_padding_phone="|6vw||6vw|false|true" custom_padding_last_edited="on|phone" animation_style="zoom" animation_intensity_zoom="8%" hover_enabled="0" border_radii="on|10px|10px|10px|10px" border_radii_image="on|4px|4px|4px|4px" box_shadow_style="preset1" box_shadow_color="rgba(110,130,208,0.18)" box_shadow_style_image="preset1" icon_hover_color="#f73145" global_colors_info="{%22gcid-e5065624-c60f-4aa9-bd26-2cf3c4270913%22:%91%22header_text_color%22%93}" icon_color__hover="RGBA(255,255,255,0)" icon_color__hover_enabled="on|hover" sticky_enabled="0"]<p>{{prof description}}</p>[/et_pb_team_member]'

VIDEO_SLIDER_DIV2 = '[et_pb_code _builder_version="4.19.0" _module_preset="default" border_radii="on|20px|20px|20px|20px" border_width_all="2px" border_color_all="#FFFFFF" border_style_all="ridge" box_shadow_style="preset1" global_colors_info="{}"][h5p id="30"][/et_pb_code]'
VIDEO_SLIDER_REPLACE2 = '[et_pb_code _builder_version="4.19.0" _module_preset="default" border_radii="on|20px|20px|20px|20px" border_width_all="2px" border_color_all="#FFFFFF" border_style_all="ridge" box_shadow_style="preset1" global_colors_info="{}"][h5p id="{{video}}"][/et_pb_code]'
FACULTY_DIV2 = '[et_pb_column type="1_3" _builder_version="4.16" custom_padding="|||" global_colors_info="{}" custom_padding__hover="|||"][et_pb_team_member name="{{professor}}" image_url="{{faculty_image}}" facebook_url="#" twitter_url="#" google_url="#" linkedin_url="#" icon_color="RGBA(255,255,255,0)" use_icon_font_size="on" icon_font_size="1px" _builder_version="4.19.0" header_font="Archivo|700|||||||" header_text_color="#544FFF" header_font_size="18px" header_line_height="1.7em" body_font="||||||||" body_line_height="1.8em" background_color="#ffffff" text_orientation="center" custom_margin="||0px|" custom_padding="30px|3.7vw|0px|3.7vw|false|true" custom_padding_tablet="|17vw||17vw|false|true" custom_padding_phone="|6vw||6vw|false|true" custom_padding_last_edited="on|phone" child_filter_saturate="110%" child_filter_brightness="105%" child_filter_contrast="120%" animation_style="zoom" animation_intensity_zoom="8%" border_radii="on|28px|28px|28px|28px" border_width_all="2px" border_radii_image="on|15px|15px|15px|15px" border_color_all_image="RGBA(255,255,255,0)" box_shadow_style="preset1" box_shadow_color="rgba(110,130,208,0.18)" box_shadow_style_image="preset1" icon_hover_color="#f73145" global_colors_info="{%22gcid-e5065624-c60f-4aa9-bd26-2cf3c4270913%22:%91%22header_text_color%22%93}" icon_color__hover="RGBA(255,255,255,0)" icon_color__hover_enabled="on|hover"]<p>{{prof description}}</p>[/et_pb_team_member][/et_pb_column]'


In [66]:
def perform_image_replacements(description, image_replacements):
    for pair in image_replacements:
        description = description.replace(pair[0], pair[1])
    
    return description

def get_search_term(term):
    return SEARCH_FORM.replace('%term', term)

def generate_sku(data_row):
    sku_list = ["-".join(data_row[col].lower().split(" ")) for col in SKU_COLS]
    return "-".join(sku_list)

def generate_product_name(data_row):
    return "-".join([data_row[col] for col in NAME_COLS])

def replace_product_videos(data_row, description):
    
    for i in PRODUCT_VIDEO_COLS:
        if not is_nan(data_row[i]):
            new_vid = VIDEO_SLIDER_REPLACE2.replace("{{video}}", str(int(data_row[i])))
            description = description.replace(VIDEO_SLIDER_DIV2, new_vid, 1)
            
    # If there are sample videos still left, remove them before generating description
    return description.replace(VIDEO_SLIDER_DIV2, "")

def replace_faculty_info(data_row, description):
    for i in range(3):
        if isinstance(data_row[FACULTY_COLS[0].format(i+1)], str):
            # Faculty Exists
            b = FACULTY_DIV2
            for j in range(3):
                d_col = data_row[FACULTY_COLS[j].format(i+1)]
                b = b.replace(get_search_term(FAC_REPLACE[j]), d_col)                
                
            description = description.replace(FACULTY_DIV2, b, 1)
      
    # Remove unused faculty divs
    return description.replace(FACULTY_DIV2, "")
    

def generate_branches(data_df, product_code):
    aff_df = data_df[data_df[PRODUCT_CODE_COL] == product_code].copy(deep=True)
    aff_df.fillna("", inplace=True)
    
    affiliations = aff_df[AFFILIATION_BRANCH_NAME].tolist()
    aff_join = ", ".join(affiliations)
    aff_join = ",and ".join(aff_join.rsplit(",", 1))
    
    if len(affiliations) > 1:
        aff_join += " branches"
    else:
        aff_join += " branch"
    
    return aff_join

def generate_product_description(description, data_row, data_df):
    product_code = None
    
    description = perform_image_replacements(description, IMAGE_REPLACEMENTS)    
    for col_name, col_value in data_row.items():
        if not isinstance(col_value, str):
            if not math.isnan(col_value):
                col_value = int(col_value)
            else:
                continue
                
        if col_name == PRODUCT_CODE_COL:
            product_code = col_value
        if col_name == AFFILIATION_BRANCH_NAME:
            col_value  = generate_branches(data_df, product_code)
            
        description = description.replace(get_search_term(col_name), str(col_value))
        
    description = replace_product_videos(data_row, description)
    description = replace_faculty_info(data_row, description)
            
    return description

def is_nan(val):
    if isinstance(val, str):
        return False
    
    return math.isnan(val)

def generate_variation_data(variation, data_row, data_df):
    
    variation[VARIATION_NAME] = variation[VARIATION_NAME].replace(get_search_term("name"), generate_product_name(data_row))
     
    if not is_nan(variation[VARIATION_SKU]):
        variation[VARIATION_SKU] = variation[VARIATION_SKU].replace(get_search_term("sku"), generate_sku(data_row))
        
    if not is_nan(variation[VARIATION_IMAGE]):
        variation[VARIATION_IMAGE] = variation[VARIATION_IMAGE].replace(get_search_term(PRODUCT_IMAGE_COL), data_row[PRODUCT_IMAGE_COL])
    
    if not is_nan(variation[VARIATION_PARENT]):
        variation[VARIATION_PARENT] = variation[VARIATION_PARENT].replace(get_search_term("sku"), generate_sku(data_row))
        
    if not is_nan(variation[VARIATION_CATEGORY]):
        variation[VARIATION_CATEGORY] = variation[VARIATION_CATEGORY].replace(get_search_term("category"), data_row["domain"])
    
    if not is_nan(variation[VARIATION_DESCRIPTION]):
        variation[VARIATION_DESCRIPTION] = generate_product_description(variation[VARIATION_DESCRIPTION], data_row, data_df)
        
    return variation

In [32]:
perform_image_replacements(variation_template.iloc[0]['Description'], IMAGE_REPLACEMENTS)
#replace_product_videos(data_df.iloc[0], variation_template.iloc[0]['Description'])

'[et_pb_section fb_built="1" _builder_version="4.19.0" _module_preset="default" background_color="gcid-6119d853-c90e-46a8-9c49-16afd63cb690" global_colors_info="{%22gcid-6119d853-c90e-46a8-9c49-16afd63cb690%22:%91%22background_color_gradient_stops%22,%22background_color%22%93,%22gcid-9e2de924-2a97-4fc3-aeaf-dd33006f3c4d%22:%91%22background_color_gradient_stops%22%93,%22gcid-9dfb48d1-038c-4d95-9042-c8cb80e2d5c6%22:%91%22background_color_gradient_stops%22%93,%22gcid-aa0f989a-f66e-4f42-9fe2-f96930f37ba6%22:%91%22background_color_gradient_stops%22%93}"][et_pb_row _builder_version="4.19.0" _module_preset="default" global_colors_info="{}"][et_pb_column type="4_4" _builder_version="4.19.0" _module_preset="default" global_colors_info="{}"][et_pb_text _builder_version="4.21.0" _module_preset="default" header_font="Archivo|600||on|||||" header_text_color="#FFFFFF" header_font_size="2.5vw" header_font_size_tablet="2.5vw" header_font_size_phone="5vw" header_font_size_last_edited="on|phone" global_

In [33]:
len(perform_image_replacements(variation_template.iloc[0]['Description'], IMAGE_REPLACEMENTS).split(FACULTY_DIV2))

#len(variation_template.iloc[0]['Description'].split(FACULTY_DIV2))

4

In [34]:
len(variation_template.iloc[0]['Description'].split(VIDEO_SLIDER_DIV2))

4

In [35]:
def generate_products(variation_df, data_df):
    generated_codes = set()
    res = []
    for idx, data_row in tqdm(data_df.head(8).iterrows(), total=len(data_df)):
        if data_row[PRODUCT_CODE_COL] not in generated_codes:    
            generated_codes.add(data_row[PRODUCT_CODE_COL])
            for ix, variation_row in variation_df.iterrows():
                res.append(generate_variation_data(variation_row, data_row, data_df))
    
    return res

def massage_types(df):
    df = df.fillna("", inplace=False)

    for tt in FLOAT_TO_INT:
        k = [int(j) if j else "" for j in df[tt] ]
        df[tt] = k
    return df

In [36]:
res = pd.DataFrame(generate_products(variation_template, data_df))
res = massage_types(res)
res.to_csv(f"{output_dir}/test_export2.csv", index=False)

  1%|▎                                        | 8/1155 [00:00<00:04, 281.97it/s]


In [268]:
generate_variation_data(variation_template.iloc[1].copy(deep=True), data_df.iloc[0].copy(deep=True), data_df)

Type                                                                       variation, virtual
SKU                                                                                       NaN
Name                                                       KCS301-Data Structures - Essential
Published                                                                                   1
Is featured?                                                                                0
                                                                          ...                
Meta: _global_colors_info                                                                 NaN
Meta: _et_builder_version                                                                 NaN
Meta: _et_pb_show_page_creation                                                           NaN
Meta: _et_pb_woo_page_content_status                                                      NaN
Meta: _et_builder_dynamic_assets_loading_attr_threshold     

In [269]:
jj = pd.read_csv("../data/product_output/test_export2.csv")
jj[GUTTER_COL].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [22]:
kc = generate_product_description(variation_template.iloc[0]['Description'], data_df.iloc[0], data_df)

/var/folders/_s/v_n5y2311rv5fm55825twr6c0000gn/T/ipykernel_12625/2980283310.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aff_df.fillna("", inplace=True)


General Strategy - First put all columns to be replaced in variation markdown and change those for the main and secondary rows. Then for all columns in data row, check if tag exists in divi blurb and change. 
For videos and images, create a special mapping since they will not exist directly in divi blurb

In [59]:
for idx, row in data_df.iterrows():
    jj = generate_sku(row)
    pp = generate_branches(data_df, row[PRODUCT_CODE_COL])
    jc = generate_product_vid_list(row)
    
jj, pp, jc

/var/folders/_s/v_n5y2311rv5fm55825twr6c0000gn/T/ipykernel_8947/3501503208.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aff_df.fillna("", inplace=True)


('bect-304-utu-bect-304',
 'Electronics & Communication Engineering, Electrical Engineering, Electrical & Electronics Engineering,and  ',
 '')

In [24]:
affiliations = data_df[data_df[PRODUCT_CODE_COL] == 'ES119'][AFFILIATION_BRANCH_NAME].tolist()
    
", ".join(affiliations[:-1]) + " and " + affiliations[-1]

' and First Year Common Course'

In [21]:
get_search_term(description_columns[0])

'{{essential_course_duration}}'

In [24]:
variation_template.columns

Index(['Type', 'SKU', 'Name', 'Published', 'Is featured?',
       'Visibility in catalog', 'Short description', 'Description',
       'Date sale price starts', 'Date sale price ends', 'Tax status',
       'Tax class', 'In stock?', 'Stock', 'Low stock amount',
       'Backorders allowed?', 'Sold individually?', 'Weight (kg)',
       'Length (cm)', 'Width (cm)', 'Height (cm)', 'Allow customer reviews?',
       'Purchase note', 'Sale price', 'Regular price', 'Categories', 'Tags',
       'Shipping class', 'Images', 'Download limit', 'Download expiry days',
       'Parent', 'Grouped products', 'Upsells', 'Cross-sells', 'External URL',
       'Button text', 'Position', 'Attribute 1 name', 'Attribute 1 value(s)',
       'Attribute 1 visible', 'Attribute 1 global',
       'Meta: _et_pb_post_hide_nav', 'Meta: _et_pb_page_layout',
       'Meta: _et_pb_side_nav', 'Meta: _et_pb_use_builder',
       'Meta: _et_pb_first_image', 'Meta: _et_pb_truncate_post',
       'Meta: _et_pb_truncate_post_date', 

In [278]:
data_df

,domain,aff_name,aff_fullname,aff_course_name,2l_course_name,aff_course_code,aff_branch_names,branch_heads,num_units,num_sections,...,Desc_Fac2,Name_Fac3,Pic_Fac3,Desc_Fac3,Name_Fac4,Pic_Fac4,Desc_Fac4,num_aff_topics,num_2l_topics,2l_coverage
0,Act2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",Data Structure,Data Structures,KCS301,Computer Science and Engineering,CS/IT,8,22,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,56.0,100%
1,Act2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",Data Structure,Data Structures,KCS301,Information Technology,CS/IT,8,22,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,56.0,100%
2,Act2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",Data Structure,Data Structures,KCS301,Computer Science and Information Technology,CS/IT,8,22,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,56.0,100%
3,Act2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",Data Structure,Data Structures,KCS301,Electronics and Computer Engineering,Electrical/Electronics,8,22,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,56.0,100%
4,Act2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",Data Structure,Data Structures,KCS301,CSE-Artificial Intelligence and Data Science,CS/IT,8,22,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,56.0,100%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,Act2Learn-B.Tech,RTMNU,Rashtrasant Tukadoji Maharaj Nagpur University...,Applied Mathematics- III,Engineering Math 3,BEENE301T,Electronics Engineering,Electrical/Electronics,6,18,...,Irphan Sir has over 18 years of teaching exper...,Atul Sir,http://test.2learn.in/wp-content/uploads/2023/...,Atul Sir has over 9 years of teaching experien...,NaN,NaN,NaN,54,NaN,NaN
2124,Act2Learn-B.Tech,RTMNU,Rashtrasant Tukadoji Maharaj Nagpur University...,Applied Mathematics- III,Engineering Math 3,BEETE301T,Electronics and Telecommunications Engineering,Electrical/Electronics,7,15,...,Irphan Sir has over 18 years of teaching exper...,Atul Sir,http://test.2learn.in/wp-content/uploads/2023/...,Atul Sir has over 9 years of teaching experien...,NaN,NaN,NaN,54,NaN,NaN
2125,Act2Learn-B.Tech,RTMNU,Rashtrasant Tukadoji Maharaj Nagpur University...,Applied Mathematics- III,Engineering Math 3,BEIT301T,Information Technology,CS/IT,6,13,...,Irphan Sir has over 18 years of teaching exper...,Atul Sir,http://test.2learn.in/wp-content/uploads/2023/...,Atul Sir has over 9 years of teaching experien...,NaN,NaN,NaN,35,NaN,NaN
2126,Act2Learn-B.Tech,RTMNU,Rashtrasant Tukadoji Maharaj Nagpur University...,Applied Mathematics- III,Engineering Math 3,BECS301T,Computer Science and Information Technology,CS/IT,5,9,...,Irphan Sir has over 18 years of teaching exper...,Atul Sir,http://test.2learn.in/wp-content/uploads/2023/...,Atul Sir has over 9 years of teaching experien...,NaN,NaN,NaN,46,NaN,NaN


## Tab Generation

In [64]:
BRANCH_REPLACE = '<h4>{0}</h4>'
URL_REPLACE = '<a href="http://test.2learn.in/{0}">{1}</a>'
delimiter = '&&'

In [73]:
def generate_aff_product_name(data_row):
    return "-".join([data_row[col] for col in [PRODUCT_CODE_COL, 'aff_course_name']])
    
def generate_tab_urls(data_df, filter_cols= ['aff_name', 'branch_heads', 'aff_branch_names']):
    generated_set = {}
    res = []
    for idx, row in tqdm(data_df.iterrows(), total =len(data_df)):
        try:
            data = [row[i] for i in filter_cols]
            gen_check = delimiter.join(data)
            if gen_check in generated_set:
                continue

            query = '&'.join([f'{filter_col} == "{row[filter_col]}"' for filter_col in filter_cols])
            subset_df = data_df.query(query)

            url_set = []
            for idd, subset_row in subset_df.iterrows():
                sku, product_name = generate_sku(subset_row), generate_aff_product_name(subset_row)
                url_set.append(URL_REPLACE.format(sku, product_name))

            generated_set[gen_check] = url_set
        except Exception as e:
            print(data)
            print(e)
            assert(e)
        
    return generated_set

def output_text(data, file_writer, delim_limit=1 , delimiter = '\n'):
    file_writer.write(data + delimiter*delim_limit)
    
def sort_custom(dict_inp, how='text', order='asc'):
    keys2 = list(dict_inp.keys())
    if how=='text':
        keys2 = sorted(keys2)
    elif how=='number':
        keys2 = sorted(keys2, key=lambda x: re.findall(r'\d+', x)[0])
        
    if order=='desc':
        keys2 = list(reversed(keys2))
    
    sorted_dict = {i: dict_inp[i] for i in keys2}
    return sorted_dict
    
def output_tab_urls(generated_set, output_file='../data/tab_url_output/output.txt'):
    
    data_map = {}
    for key in generated_set.keys():
        aff_name, branch_group, aff_branch_name = key.split(delimiter)
        
        val = data_map.get(aff_name, {})
        if branch_group not in val:
            val[branch_group] = {}
        
        val[branch_group][aff_branch_name] = generated_set[key]
        
        data_map[aff_name] = val
        
    for aff in data_map:
        data_map[aff] = sort_custom(data_map[aff])
        # Sort branch group in ascending order
        for group in data_map[aff]:
            data_map[aff][group] = sort_custom(data_map[aff][group])
            # Sort branches by digit
            for bb in data_map[aff][group]:
                try:
                    data_map[aff][group][bb] = sorted(data_map[aff][group][bb], key=lambda x: re.findall(r'\d+', x)[1])
                except:
                    print(data_map[aff][group][bb])
                    continue
                    
    with open(output_file, 'w') as f:
        for aff_name, data in data_map.items():
            output_text(aff_name, f)
            for group, group_data in data.items():
                output_text(group, f, delim_limit=2)
                for aff_branch_name, branch_data in group_data.items():
                    aff_branch_format = BRANCH_REPLACE.format(aff_branch_name)
                    output_text(aff_branch_format, f, delim_limit=2)
                    
                    for url in branch_data: output_text(url, f, delim_limit=1)
                
                    output_text('',f)
                output_text('', f, delim_limit=2)
            output_text('', f, delim_limit=2)
            
    return None

output_tab_urls(generate_tab_urls(data_df))

100%|█████████████████████████████████████| 1086/1086 [00:00<00:00, 4850.84it/s]

['<a href="http://test.2learn.in/mtu-mtucsdbms">MTUCSDBMS-Database Management Systems</a>', '<a href="http://test.2learn.in/mtu-mtucsde">MTUCSDE-Digital Electronics</a>', '<a href="http://test.2learn.in/mtu-mtucsdm">MTUCSDM-Discrete Maths</a>', '<a href="http://test.2learn.in/mtu-mtucsds">MTUCSDS-Data Structures</a>', '<a href="http://test.2learn.in/mtu-mtucsos">MTUCSOS-Operating System</a>', '<a href="http://test.2learn.in/mtu-mtucstoc">MTUCSTOC-Theory of Computation</a>']
['<a href="http://test.2learn.in/mtu-mtucebmc">MTUCEBMC-Building Material and Construction</a>', '<a href="http://test.2learn.in/mtu-mtucefm">MTUCEFM-Fluid Mechanics </a>', '<a href="http://test.2learn.in/mtu-mtucercc">MTUCERCC-RCC</a>', '<a href="http://test.2learn.in/mtu-mtucesm">MTUCESM-Soil Mechanics</a>', '<a href="http://test.2learn.in/mtu-mtucesom">MTUCESOM-Strength of Material</a>', '<a href="http://test.2learn.in/mtu-mtucesv">MTUCESV-Surveying</a>']
['<a href="http://test.2learn.in/mtu-mtuccbee">MTUCCBEE-Ba

In [ ]:
data_df.groupby(['aff_name','aff_branch_names']).get_group(("aff_branch_names",'Information Technology'))

KeyError: ('aff_branch_names', 'Information Technology')

In [69]:
data_df.query('aff_name == "AKTU" & aff_course_name == "Data Structure"')

,domain,aff_name,aff_fullname,aff_course_code,aff_course_name,2l_course_name,aff_branch_names,branch_heads,num_units,num_sections,...,Pic_Fac4,Desc_Fac4,num_aff_topics,num_2l_topics,2l_coverage,Syl_Img,Topics,Unnamed: 54,s1,s2
84,2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",KCS301,Data Structure,Data Structures,CSE-Artificial Intelligence,CS/IT,6,20,...,NaN,NaN,56,56,100%,https://sanats.s3.us-west-2.amazonaws.com/Syll...,"Introduction: Basic Terminology, *Elementary D...",NaN,https://sanats.s3.us-west-2.amazonaws.com/Syll...,https://sanats.s3.us-west-2.amazonaws.com/Syll...
85,2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",KCS301,Data Structure,Data Structures,CSE-Artificial Intelligence and Data Science,CS/IT,6,20,...,NaN,NaN,56,56,100%,https://sanats.s3.us-west-2.amazonaws.com/Syll...,"Introduction: Basic Terminology, *Elementary D...",NaN,https://sanats.s3.us-west-2.amazonaws.com/Syll...,https://sanats.s3.us-west-2.amazonaws.com/Syll...
86,2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",KCS301,Data Structure,Data Structures,CSE-Artificial Intelligence and Machine Learning,CS/IT,6,20,...,NaN,NaN,56,56,100%,https://sanats.s3.us-west-2.amazonaws.com/Syll...,"Introduction: Basic Terminology, *Elementary D...",NaN,https://sanats.s3.us-west-2.amazonaws.com/Syll...,https://sanats.s3.us-west-2.amazonaws.com/Syll...
87,2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",KCS301,Data Structure,Data Structures,CSE-Data Science,CS/IT,6,20,...,NaN,NaN,56,56,100%,https://sanats.s3.us-west-2.amazonaws.com/Syll...,"Introduction: Basic Terminology, *Elementary D...",NaN,https://sanats.s3.us-west-2.amazonaws.com/Syll...,https://sanats.s3.us-west-2.amazonaws.com/Syll...
88,2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",KCS301,Data Structure,Data Structures,CSE-Internet of Things,CS/IT,6,20,...,NaN,NaN,56,56,100%,https://sanats.s3.us-west-2.amazonaws.com/Syll...,"Introduction: Basic Terminology, *Elementary D...",NaN,https://sanats.s3.us-west-2.amazonaws.com/Syll...,https://sanats.s3.us-west-2.amazonaws.com/Syll...
89,2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",KCS301,Data Structure,Data Structures,Computer Engineering,CS/IT,6,20,...,NaN,NaN,56,56,100%,https://sanats.s3.us-west-2.amazonaws.com/Syll...,"Introduction: Basic Terminology, *Elementary D...",NaN,https://sanats.s3.us-west-2.amazonaws.com/Syll...,https://sanats.s3.us-west-2.amazonaws.com/Syll...
90,2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",KCS301,Data Structure,Data Structures,Computer Science,CS/IT,6,20,...,NaN,NaN,56,56,100%,https://sanats.s3.us-west-2.amazonaws.com/Syll...,"Introduction: Basic Terminology, *Elementary D...",NaN,https://sanats.s3.us-west-2.amazonaws.com/Syll...,https://sanats.s3.us-west-2.amazonaws.com/Syll...
91,2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",KCS301,Data Structure,Data Structures,Computer Science-Software Engineering,CS/IT,6,20,...,NaN,NaN,56,56,100%,https://sanats.s3.us-west-2.amazonaws.com/Syll...,"Introduction: Basic Terminology, *Elementary D...",NaN,https://sanats.s3.us-west-2.amazonaws.com/Syll...,https://sanats.s3.us-west-2.amazonaws.com/Syll...
92,2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",KCS301,Data Structure,Data Structures,Computer Science and Design,CS/IT,6,20,...,NaN,NaN,56,56,100%,https://sanats.s3.us-west-2.amazonaws.com/Syll...,"Introduction: Basic Terminology, *Elementary D...",NaN,https://sanats.s3.us-west-2.amazonaws.com/Syll...,https://sanats.s3.us-west-2.amazonaws.com/Syll...
93,2Learn-B.Tech,AKTU,"Dr A.P.J Abdul Kalam Technical University, Luc...",KCS301,Data Structure,Data Structures,Computer Science and Engineering,CS/IT,6,20,...,NaN,NaN,56,56,100%,https://sanats.s3.us-west-2.amazonaws.com/Syll...,"Introduction: Basic Terminology, *Elementary D...",NaN,https://sanats.s3.us-west-2.amazonaws.com/Syll...,http

In [226]:
data_df[(data_df['aff_name'] == 'AKTU') & (data_df['aff_course_name']=='Data Structure')]

,domain,aff_name,aff_course_name,2l_course_name,aff_course_code,aff_branch_names,num_units,num_sections,num_topics,num_alm,...,Desc_Fac2,Name_Fac3,Pic_Fac3,Desc_Fac3,Name_Fac4,Pic_Fac4,Desc_Fac4,num_aff_topics,num_2l_topics,2l_coverage
0,Act2Learn-B.Tech,AKTU,Data Structure,Data Structures,KCS301,Computer Science and Engineering,8,22,59,169,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN
1,Act2Learn-B.Tech,AKTU,Data Structure,Data Structures,KCS301,Information Technology,8,22,59,169,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN
2,Act2Learn-B.Tech,AKTU,Data Structure,Data Structures,KCS301,Computer Science and Information Technology,8,22,59,169,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN
3,Act2Learn-B.Tech,AKTU,Data Structure,Data Structures,KCS301,Electronics and Computer Engineering,8,22,59,169,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN
4,Act2Learn-B.Tech,AKTU,Data Structure,Data Structures,KCS301,CSE-Artificial Intelligence and Data Science,8,22,59,169,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN
5,Act2Learn-B.Tech,AKTU,Data Structure,Data Structures,KCS301,CSE-Artificial Intelligence and Machine Learning,8,22,59,169,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN
6,Act2Learn-B.Tech,AKTU,Data Structure,Data Structures,KCS301,CSE-Computer Science and Design,8,22,59,169,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN
7,Act2Learn-B.Tech,AKTU,Data Structure,Data Structures,KCS301,CSE-Artificial Intelligence,8,22,59,169,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN
8,Act2Learn-B.Tech,AKTU,Data Structure,Data Structures,KCS301,CSE-Data Science,8,22,59,169,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN
9,Act2Learn-B.Tech,AKTU,Data Structure,Data Structures,KCS301,CSE-Internet of Things,8,22,59,169,...,Ivy Mam has over 8 years of teaching experienc...,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,NaN


In [70]:
data_df.aff_name.value_counts()

AKTU      262
RTU       143
BTU       142
GGSIPU    140
RGPV      136
RTMNU     116
SPPU      113
MTU        34
Name: aff_name, dtype: int64

## Get URL Check For Syllabus

In [72]:
data_df2 = pd.read_csv('../data/courses-data-btech-v4.csv')
data_df2.aff_name.value_counts()

AKTU      262
RTU       143
BTU       142
GGSIPU    140
RGPV      136
RTMNU     116
SPPU      113
MTU        34
Name: aff_name, dtype: int64

In [56]:
import requests

def get_url_status(url):
    try:
        response = requests.get(url)
        return response.status_code
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# Example usage

res = {}
for i in tqdm(set(data_df2['s2'])):
    res[i] =  get_url_status(i)

100%|█████████████████████████████████████████| 714/714 [08:12<00:00,  1.45it/s]


In [57]:
from collections import Counter
Counter(res.values())

Counter({200: 701, 403: 13})

In [58]:
missing_urls = [i for i in res if res[i]== 403]
missing_urls

['https://sanats.s3.us-west-2.amazonaws.com/Syllabus/B.Tech/GGSIPU_ETCE309_Soil_Mechanics_Syllabus.png',
 'https://sanats.s3.us-west-2.amazonaws.com/Syllabus/B.Tech/AKTU_KEE302_Electrical_Measurements_Instrumentation_Syllabus.webp',
 'https://sanats.s3.us-west-2.amazonaws.com/Syllabus/B.Tech/RTU_3MH406_Engineering_Thermodynamics_Syllabus.png',
 'https://sanats.s3.us-west-2.amazonaws.com/Syllabus/B.Tech/GGSIPU_ETIT309_Communication_Systems_Syllabus.png',
 'https://sanats.s3.us-west-2.amazonaws.com/Syllabus/B.Tech/RTMNU_BES11T_Applied_Mathematics-I_Syllabus.webp',
 'https://sanats.s3.us-west-2.amazonaws.com/Syllabus/B.Tech/RTMNU_BEEE402T_Digital_Electronics_Syllabus.png',
 'https://sanats.s3.us-west-2.amazonaws.com/Syllabus/B.Tech/RTMNU_BES21T_Mathematics-II_Syllabus.webp',
 'https://sanats.s3.us-west-2.amazonaws.com/Syllabus/B.Tech/AKTU_KCE401_Materials,_Testing_Construction_Practices_Syllabus.webp',
 'https://sanats.s3.us-west-2.amazonaws.com/Syllabus/B.Tech/GGSIPU_ETEC303_Digital_Comm

In [13]:
import boto3
from dotenv import load_dotenv

In [14]:
load_dotenv(boto_config_file)

True

In [22]:
def get_s3_file_list(s3_client, bucket_name, folder_prefix):
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder_prefix)

    file_list = []
    for obj in response['Contents']:
        file_list.append(obj['Key'])

    return file_list

# Create a Boto client or resource object using the configuration
s3_client = boto3.client('s3')

# Provide your bucket name and folder prefix
bucket_name = 'sanats'
folder_prefix = 'Syllabus/B.Tech/'

# Call the function to get the file list
files = get_s3_file_list(s3_client, bucket_name, folder_prefix)
len(files)

718

In [29]:
url_portion = [i.split('/')[-1] for i in files][1:]

In [59]:
#!pip install python-Levenshtein
import Levenshtein

def closest_dist(url, url_list):
    val = min(range(len(url_list)), key=lambda i: Levenshtein.distance(url, url_list[i]))
    return url_list[val]


missing_urls_m = [i.split('/')[-1] for i in missing_urls]
closest_url = [closest_dist(i, url_portion) for i in missing_urls_m]

In [60]:
missing_df = pd.DataFrame(list(zip(missing_urls_m, closest_url)), columns = ['Missing URL Text', 'S3 Text'])
missing_df

,Missing URL Text,S3 Text
0,GGSIPU_ETCE309_Soil_Mechanics_Syllabus.png,GGSIPU_ETCE309_Soil_Mechanics_Syllabus.webp
1,AKTU_KEE302_Electrical_Measurements_Instrument...,AKTU_KEE302_Electrical_Measurements___Instrume...
2,RTU_3MH406_Engineering_Thermodynamics_Syllabus...,RTU_3MH406_Engineering_Thermodynamics_Syllabus...
3,GGSIPU_ETIT309_Communication_Systems_Syllabus.png,GGSIPU_ETIT309_Communication_Systems_Syllabus....
4,RTMNU_BES11T_Applied_Mathematics-I_Syllabus.webp,RTMNU_BSE11T_Applied_Mathematics-I_Syllabus.webp
5,RTMNU_BEEE402T_Digital_Electronics_Syllabus.png,RTMNU_BEEE402T_Digital_Electronics_Syllabus.webp
6,RTMNU_BES21T_Mathematics-II_Syllabus.webp,RTMNU_BSE21T_Mathematics-II_Syllabus.webp
7,"AKTU_KCE401_Materials,_Testing_Construction_Pr...","AKTU_KCE401_Materials,_Testing___Construction_..."
8,GGSIPU_ETEC303_Digital_Communication_Syllabus.png,GGSIPU_ETEC303_Digital_Communication_Syllabus....
9,BTU_3MH406_Engineering_Thermodynamics_Syllabus...,BTU_3MH406_Engineering_Thermodynamics_Syllabus...


In [61]:
missing_df.to_csv('../data/s3_url_mismatch.csv', index=False)

# 